# Contents
- [Data-Setup](#Data-Setup)
- [Randomization Methods](#Randomization-Methods)
- [AA Testing](#AA-Testing)
    - [SRM Check](#SRM-Check)
- [Power Analysis](#Power-Analysis)
- [AB Testing](#AB-Testing)
    - [Definitions](#Definitions)
    - [Results Summary](#Results-Summary)
    - [Test](#Test)
- [How Long](#How-Long)
- [Results](#Results)
    - [Summary](#Conversion-Rates)
    - [Visualization](#Visualization)
    - [Confidence Intervals Within Groups](#Confidence-Intervals-Outcomes)
    - [Confidence Intervals Across Groups](#Confidence-Intervals-Difference)
    - [Conclusion](#Conclusion)
- [Post Hoc Analysis](#Post-Hoc-Analysis)
    - [Segmented Lift](#Segmented-Lift)
    - [Gaurdrail Metrics](#Gaurdrail-Metrics)
    - [Rollout Simulation](#Rollout-Simulation)
- [Other Notes](#Other-Notes)
    - [Experimentation-Infrastructure](#Experimentation-Infrastructure)
- [Scratch Notes](#Scratch-Notes)
    - [Other Use Cases](#Other-Use-Cases)

# Data Setup

In [2]:
# Display all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import ttest_ind, chi2_contingency, ttest_ind, ttest_rel, mannwhitneyu
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.power import TTestIndPower

# Set Seed 
my_seed=42
np.random.seed(my_seed)

##### Sample user data

In [9]:
observations_count = 100 

np.random.seed(my_seed)  # For reproducibility
users = pd.DataFrame({
    'user_id': range(1, observations_count+1),
    'platform': np.random.choice(['iOS', 'Android'], size=observations_count, p=[0.6, 0.4]),  # 60% iOS, 40% Android
    'engagement_score': np.random.normal(50, 15, observations_count),  # Simulated user engagement scores
    'city': np.random.choice(['ny', 'sf', 'chicago', 'austin'], size=observations_count),
    'past_purchase_count': np.random.normal(loc=50, scale=10, size=observations_count)  # pre_experiment_metric for CUPED randomization
})
users




,user_id,platform,engagement_score,city,past_purchase_count
0,1,iOS,51.305706,sf,46.847308
1,2,Android,45.514890,ny,57.589692
2,3,Android,51.376412,sf,42.271748
3,4,iOS,20.186466,ny,47.631814
4,5,iOS,46.704922,austin,45.146365
...,...,...,...,...,...
95,96,iOS,37.762846,austin,47.978073
96,97,iOS,48.843474,austin,47.823188
97,98,iOS,55.117280,sf,60.987769
98,99,iOS,54.150362,austin,58.254163


[Back to the top](#Contents)
___


# Randomization Methods

- Ensures that differences in outcome metrics are due to the experiment and not pre-existing differences between users.
- Eliminates selection bias (e.g., users choosing their own group).
- Helps balance confounding variables (e.g., demographics, device type, purchase history).
- Enables valid statistical inference.

In [10]:
randomization_method = "simple"

#### Simple Randomization
- Each user has an equal chance (e.g., 50/50 split for A/B) of being assigned to treatment or control.
- Works well when sample sizes are large.

In [4]:
# Assign each user to 'control' or 'treatment' with 50% probability
users['group'] = np.random.choice(['control', 'treatment'], size=len(users), p=[0.5, 0.5])

# Display the result
users

,user_id,group
0,1,control
1,2,treatment
2,3,treatment
3,4,treatment
4,5,control
...,...,...
95,96,control
96,97,treatment
97,98,control
98,99,control


#### Stratified Sampling
- Ensures balance across key segments (e.g., country, platform, user tenure).
- Example: If 60% of your users are on iOS and 40% on Android, simple randomization might cause an imbalance, so you stratify by platform.

In [ ]:
# Stratify by platform to ensure balance
train, test = train_test_split(users, test_size=0.5, stratify=users['platform'], random_state=my_seed)

# Assign groups
train['group'] = 'control'
test['group'] = 'treatment'

# Merge and display
users = pd.concat([train, test]).sort_values('user_id')
users

#### Block Randomization

In [ ]:
# Define block size (e.g., groups of 10 users)
block_size = 10

# Assign blocks
users['block'] = (users['user_id'] - 1) // block_size

# Within each block, randomly assign 50% to control and 50% to treatment
users['group'] = users.groupby('block')['user_id'].transform(lambda x: np.random.choice(['control', 'treatment'], size=len(x), replace=True))

# Drop the block column after assignment
# users = users.drop(columns=['block'])
users

#### Match Pair Randomization

Participants are paired based on similar characteristics before being randomly assigned to different groups. This ensures that treatment and control groups are balanced on key covariates, reducing variance and improving statistical power.

When to Use Matched-Pair Randomization?
- When you have a small sample size and need to control for confounders.
- When key characteristics (e.g., age, income, purchase history) could influence the outcome.
- When you want to minimize variance by ensuring similar individuals are in each group.

How It Works:
- Identify key variables that might impact the outcome (e.g., age, income, engagement level).
- Sort users based on these variables.
= Create pairs (or small groups) of users with similar characteristics.
- Within each pair, randomly assign one user to treatment and the other to control.


In [ ]:
# Sort users by engagement to create pairs
users = users.sort_values(by='engagement_score').reset_index(drop=True)

# Assign treatment/control in pairs
users['group'] = np.where(users.index % 2 == 0, 'control', 'treatment')
users


#### Cluster Randomization
- Assigns whole groups (e.g., entire cities or schools) instead of individuals.
- Useful when spillover effects are a concern (e.g., referral programs).

In [ ]:
    # Randomly assign cities to control or treatment
unique_cities = users['city'].unique()
city_assignments = dict(zip(unique_cities, np.random.choice(['control', 'treatment'], size=len(unique_cities), replace=True)))

# Assign users based on their city
users['group'] = users['city'].map(city_assignments)
users

#### CUPED
- Uses historical data to reduce variance and increase test sensitivity.
- Doesn’t change how users are assigned but helps in post-analysis.

In [ ]:
# Simple Randomization
users['group'] = np.random.choice(['control', 'treatment'], size=len(users), p=[0.5, 0.5])


pre_experiment_metric = 'past_purchase_count'

# Compute CUPED adjustment
X = users[[pre_experiment_metric]]
X = sm.add_constant(X)  # Add intercept
y = np.random.normal(loc=0, scale=5, size=observation_count)  # Simulated experiment outcome

# Regression to estimate theta (correction factor)
theta = sm.OLS(y, X).fit().params[pre_experiment_metric]

# Adjust the outcome using pre-experiment data
users['adjusted_outcome'] = y - theta * users[pre_experiment_metric]
users

[Back to the top](#Contents)
___


# AA Testing

# What specifically are we checking for???????

A/A testing is an experiment where both groups (control and treatment) receive the **same experience** to ensure that randomization is working correctly. It acts as a **sanity check** before running an A/B test.

##### Importance
- **Validates Randomization:** Ensures that groups are statistically similar before testing.
- **Detects Sample Ratio Mismatch (SRM):** Verifies that user assignment is balanced.
- **Estimates Variance for Power Analysis:** Helps understand variability before defining sample size for A/B testing.
- **Checks for Pre-Existing Bias:** Ensures no systematic differences exist between control and treatment groups.

##### Process
- **Randomly assign users** to two equal groups (just like an A/B test).
- **Measure key metrics** (e.g., conversion rate, engagement, revenue).
- **Perform statistical tests** (e.g., t-test for continuous data, chi-square for categorical data) to confirm no significant difference.
- **Analyze Sample Ratio Mismatch (SRM)** to verify even split between groups.

##### Result
- **No significant difference:** Randomization is working correctly, and the experiment is set up properly.
- **Significant difference detected:** Investigate potential issues such as randomization bugs, sample bias, or instrumentation errors.


#### 🧪 When to Use Which Statistical Test in A/B Testing

| **Metric Type**        | **Example**                        | **Recommended Test**                      | **Why**                                                  |
|------------------------|------------------------------------|-------------------------------------------|-----------------------------------------------------------|
| Continuous             | Revenue, time on site, scores      | `scipy.stats.ttest_ind` (T-test)          | Compares means of two independent groups                 |
| Continuous (unequal variance) | Same as above               | `ttest_ind(..., equal_var=False)`         | Welch’s T-test — safer when variances differ             |
| Binary (0/1 outcomes)  | Conversion, click, purchase        | `statsmodels.stats.proportions_ztest`     | Compares proportions between two groups                  |
| Count data             | # pageviews, # items bought        | Poisson or Negative Binomial test         | For skewed count distributions                           |
| Non-parametric         | Ordinal/skewed data, NPS scores    | Mann-Whitney U test                       | No assumption of normality                               |
| Multiple groups (A/B/C)| Multi-variant tests                | ANOVA (continuous), Chi-squared (binary)  | Tests across 3+ groups                                   |

---

#### ✅ Quick Rules of Thumb:
- If your metric is **continuous + normal-ish** → Use **T-test**
- If it’s **binary (e.g., clicked or not)** → Use **Z-test**
- If it’s **non-normal or skewed** → Use **Mann-Whitney U test**
- If testing **3 or more variants** → Use **ANOVA** or **Chi-squared**

In [ ]:
np.random.seed(my_seed)

# Step 1: Simulate a base population of customers
n_customers = 2000
population = pd.DataFrame({
    'user_id': np.arange(1, n_customers + 1),
    'is_eligible': np.random.choice([0, 1], size=n_customers, p=[0.4, 0.6]),  # 60% eligible
    'engagement_score': np.random.normal(loc=50, scale=10, size=n_customers)  # some behavioral metric
})
print("population:\n")
population

# Step 2: Filter to eligible population
eligible_population = population[population['is_eligible'] == 1].copy()
n_eligible = len(eligible_population)

# Step 3: Randomly assign eligible users into two groups (A1 and A2)
eligible_population['group'] = np.random.choice(['A1', 'A2'], size=n_eligible, replace=True)
print("eligible_population:\n")
eligible_population

# Step 4: Split into the two groups
group_A1 = eligible_population[eligible_population['group'] == 'A1']['engagement_score']
group_A2 = eligible_population[eligible_population['group'] == 'A2']['engagement_score']
print("group_A1, group_A2:\n")
print(group_A1.head())
print(group_A2.head())

## SRM Check 
Is group assignment balanced?

- 🔍 SRM (Sample Ratio Mismatch) checks whether the observed group sizes match the expected ratio.
- In a perfect world, random assignment to 'A1' and 'A2' should give ~50/50 split.
- SRM helps catch bugs in randomization, data logging, or user eligibility filtering.

🎯 Real-World Experiment Split Ratios

| **Scenario**                     | **Split**              | **Why**                                 |
|----------------------------------|------------------------|------------------------------------------|
| Default A/B                      | 50 / 50                | Maximizes power and ensures fairness     |
| Risky feature                    | 10 / 90 or 20 / 80     | Limits user exposure to minimize risk    |
| Ramp-up                          | Step-wise (1-5-25-50…) | Gradual rollout to catch issues early    |
| A/B/C Test                       | 33 / 33 / 33 or weighted | Compare multiple variants fairly or with bias |
| High control confidence needed   | 70 / 30 or 60 / 40     | More stability in baseline comparisons   |


In [ ]:
observed_counts = eligible_population['group'].value_counts().sort_index()
expected_counts = [n_eligible / 2, n_eligible / 2]

# Print observed counts and percentages
print("\n📊 Group Assignment Breakdown")
for group in observed_counts.index:
    count = observed_counts[group]
    pct = count / n_eligible * 100
    print(f"Group {group}: {count} users ({pct:.2f}%)")
    
# Chi-Square Goodness of Fit Test
chi2_stat, chi2_p = stats.chisquare(f_obs=observed_counts, f_exp=expected_counts)

print("\n🔍 SRM Check")
print(f"Chi2 Stat: {chi2_stat:.4f}")
print(f"P-value : {chi2_p:.4f}")
if chi2_p < 0.05:
    print("⚠️ Sample Ratio Mismatch detected — investigate assignment logic.")
else:
    print("✅ No SRM — group assignment is balanced.")

In [ ]:
# Step 6: Run 2-sample (independent) t-test
t_stat, p_value = stats.ttest_ind(group_A1, group_A2)

print(f"\nT-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")
if p_value < 0.05:
    print("⚠️ Statistically significant difference found — check randomization.")
else:
    print("✅ No significant difference — randomization looks good.")

# Step 7: Visualize distributions
plt.hist(group_A1, bins=30, alpha=0.5, label='Group A1');
plt.hist(group_A2, bins=30, alpha=0.5, label='Group A2');
plt.title('AA Test: Distribution Comparison');
plt.legend();
plt.show();

[Back to the top](#Contents)
___


# Power Analysis

Power analysis helps determine the **minimum sample size** needed to detect an expected effect with statistical confidence.

`Why It Matters:`
- Avoids **underpowered** tests (can't detect real differences)
- Avoids **overpowered** tests (wastes resources)
- Balances tradeoffs between **sample size**, **effect size**, **confidence level**, and **statistical power**

`Key Inputs:`
- **alpha (α):** Significance level (probability of Type I error, usually 0.05)
- **Power (1 - β):** Probability of detecting a true effect (commonly 0.80 or 0.90)
- **Baseline conversion rate:** Current performance (e.g., 10%)
- **Minimum Detectable Effect (MDE):** Smallest lift you care to detect (e.g., +2%)

We use a two-sample z-test for proportions to estimate sample size per group.

In [ ]:
def calculate_sample_size(alpha, power, baseline_rate, mde):
    """
    Calculate the sample size required for each group in an A/B test.
    
    Parameters:
    - alpha: Significance level (e.g., 0.05)
    - power: Desired power (e.g., 0.80)
    - baseline_rate: Baseline conversion rate (as a proportion, e.g., 0.10)
    - mde: Minimum Detectable Effect (as a proportion, e.g., 0.02 for +2%)

    Returns:
    - Sample size per group
    """
    # Z-scores
    z_alpha = stats.norm.ppf(1 - alpha / 2)
    z_beta = stats.norm.ppf(power)

    # Pooled standard error
    pooled_std = np.sqrt(
        2 * baseline_rate * (1 - baseline_rate) + 
        mde**2 / 2  # adjustment for effect size difference
    )

    # Sample size formula
    n = ((z_alpha + z_beta)**2 * 2 * baseline_rate * (1 - baseline_rate)) / (mde**2)
    return int(np.ceil(n))

In [ ]:
# Inputs
alpha = 0.05
power = 0.80
baseline_rate = 0.10  # 10% baseline conversion
mde = 0.02            # 2% lift

required_sample_size = calculate_sample_size(alpha, power, baseline_rate, mde)

# Conclusions
print("📈 Power Analysis Summary")
print(f"- Baseline conversion rate: {baseline_rate}")
print(f"- Minimum detectable effect (MDE): {mde}")
print(f"- Target conversion rate: {(baseline_rate + mde):.2f}")
print(f"- Significance level (α): {alpha}")
print(f"- Statistical power (1 - β): {power}")

print(f"\n✅ Result:")
print(f"To detect a lift from {baseline_rate} to {(baseline_rate + mde):.2f} with")
print(f"{power*100:.0f}% power and {alpha*100:.0f}% significance level,")
print(f"you need at least {required_sample_size} users in each group (control and treatment).")
print(f"Total required sample size: {required_sample_size * 2} users.\n")

[Back to the top](#Contents)
___

# AB Testing

### 🛠️ Metric Tracked:
- **Primary metric:** Conversion rate (binary: clicked = 1, did not click = 0)
- **Unit of analysis:** Unique user

---

#### 📈 Outcome Analysis Plan:
- Two-sample **z-test for proportions** to compare conversion rates
- Compute confidence intervals and p-values
- Optional: visualizations of effect size and confidence bounds

In [ ]:
np.random.seed(my_seed)

# Step 1: Simulate eligible population
n = 7064  # from power analysis
users = pd.DataFrame({
    'user_id': np.arange(1, n + 1),
})

# Step 2: Randomly assign users to control or treatment
users['group'] = np.random.choice(['control', 'treatment'], size=n, replace=True)
users

In [ ]:
# Step 3: Simulate conversions
# Assume baseline = 0.10, treatment = 0.12
conversion_rate = {
    'control': 0.10,
    'treatment': 0.12
}
users['converted'] = users['group'].apply(lambda g: np.random.binomial(1, conversion_rate[g]))
users

In [ ]:
# Step 4: View summary
summary = users.groupby('group')['converted'].agg(['count', 'sum', 'mean']).rename(columns={
    'count': 'n_users',
    'sum': 'n_converted',
    'mean': 'conversion_rate'
})
summary

In [ ]:
# Step 5: Run 2-proportion z-test
control_conv = summary.loc['control', 'conversion_rate']
treatment_conv = summary.loc['treatment', 'conversion_rate']
n_control = summary.loc['control', 'n_users']
n_treatment = summary.loc['treatment', 'n_users']
x_control = summary.loc['control', 'n_converted']
x_treatment = summary.loc['treatment', 'n_converted']

# Pooled conversion rate
p_pooled = (x_control + x_treatment) / (n_control + n_treatment)
se_pooled = np.sqrt(p_pooled * (1 - p_pooled) * (1/n_control + 1/n_treatment))

z_stat = (treatment_conv - control_conv) / se_pooled
p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))

[Back to the top](#Contents)
___


# Results

#### Conversion Rates

In [ ]:
print(f"\n📊 Z-Test Results")
print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value    : {p_value:.4f}")
if p_value < 0.05:
    print("✅ Statistically significant difference detected.")
else:
    print("🚫 No significant difference detected.")

In [ ]:
# Already generated in earlier step, but good to reinforce
summary

#### Visualization

In [ ]:
# Plot bar chart
fig, ax = plt.subplots()
bars = ax.bar(['Control', 'Treatment'],
              [control_conv, treatment_conv],
              color=['gray', 'skyblue'])

# Add values on top
for bar in bars:
    height = bar.get_height();
    ax.annotate(f'{height:.4f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 5),  # vertical offset
                textcoords="offset points",
                ha='center', va='bottom');
ax.set_ylabel('Conversion Rate');
ax.set_title('A/B Test: Conversion Rate by Group');
ax.set_ylim(0, max(control_conv, treatment_conv) + 0.02);
ax.grid(axis='y', linestyle='--', alpha=0.7);
plt.show();

#### 95% Confidence Intervals (`outcome of group`)

- The 95% confidence interval gives a range in which we expect the **true conversion rate** to fall for each group.
- If the confidence intervals **do not overlap**, it's strong evidence that the difference is statistically significant.
- If they **do overlap**, it doesn't guarantee insignificance — you still need the p-value to decide — but it suggests caution when interpreting lift.

In [ ]:
# Compute confidence intervals
def compute_ci(p, n, z=1.96):
    se = np.sqrt(p * (1 - p) / n)
    return (p - z*se, p + z*se)

ci_control = compute_ci(control_conv, n_control)
ci_treatment = compute_ci(treatment_conv, n_treatment)

# Plot with error bars
plt.errorbar(['Control', 'Treatment'],
             [control_conv, treatment_conv],
             yerr=[[control_conv - ci_control[0], treatment_conv - ci_treatment[0]],
                   [ci_control[1] - control_conv, ci_treatment[1] - treatment_conv]],
             fmt='o', capsize=10, color='black')
plt.ylabel('Conversion Rate');
plt.title('Conversion Rate with 95% Confidence Intervals');
plt.grid(axis='y', linestyle='--', alpha=0.7);
plt.show();

#### 95% Confidence Intervals (`difference in outcomes`). AKA `Lift Analysis`

In [ ]:
# Calculate absolute lift
lift = treatment_conv - control_conv

# Standard error for the difference in proportions
se_diff = np.sqrt(
    (control_conv * (1 - control_conv) / n_control) +
    (treatment_conv * (1 - treatment_conv) / n_treatment)
)

# 95% Confidence Interval for the lift
z = 1.96  # for 95%
ci_lower = lift - z * se_diff
ci_upper = lift + z * se_diff

# Print result
print("📈 Lift Analysis")
print(f"- Absolute Lift: {lift:.4f}")
print(f"- 95% Confidence Interval for Lift: [{ci_lower:.4f}, {ci_upper:.4f}]")

# Interpretation
if ci_lower > 0:
    print("✅ We're 95% confident the new version improved conversion.")
elif ci_upper < 0:
    print("🚫 We're 95% confident the new version hurt conversion.")
else:
    print("🤷 The confidence interval includes 0 — we can't say the lift is statistically significant.")


#### Final Conclusion

In [ ]:
print("" + "="*40)
print("          📊FINAL A/B TEST SUMMARY")
print("="*40)

print(f"👥  Control conversion rate   :  {control_conv:.4f}")
print(f"🧪  Treatment conversion rate :  {treatment_conv:.4f}")
print(f"📈  Absolute lift             :  {(treatment_conv - control_conv):.4f}")
print(f"📊  Percentage lift           :  {((treatment_conv - control_conv)/control_conv):.2%}")
print(f"🧪  P-value (from z-test)     :  {p_value:.4f}")

print("-" * 40)

if p_value < 0.05:
    print("✅  RESULT: Statistically significant improvement detected.")
else:
    print("❌  RESULT: No statistically significant difference detected.")

print("="*40 + "\n")

[Back to the top](#Contents)
___


# How Long
##### to run the test?


The runtime of an A/B test depends on how quickly you can reach the required **sample size per group**, as estimated during power analysis.

##### Key Inputs:
- ✅ Daily eligible traffic volume
- ✅ Required sample size (from power analysis)
- ✅ Whether traffic is split 50/50 or unevenly

##### Formula:
> **Days = Required Sample Size per Group / (Daily Eligible Users × Group Split Proportion)**

This ensures the experiment runs **long enough to detect the effect** with the desired statistical confidence.

##### ⏳ How Long to Run an A/B Test — Real-World Guide

1. **Estimate sample size** needed (done via power analysis)
2. **Understand your traffic** — how many eligible users per day?
3. **Factor in group split** — if it's 50/50, each group gets half the traffic
4. **Divide required sample per group by daily users per group** to estimate days

---

##### 💡 Real-World Recommendations

- ✅ **Ramp-Up Period**: Don’t go full traffic on Day 1. Start with 5%, then 25%, then 50% over 2–3 days. This is safer and helps catch bugs early.
  
- ✅ **Cool-Down Buffer**: Let the test stabilize before analysis. Avoid cutting off during weekends or anomalies.
  
- ✅ **Trust Checks**:
  - Run an **A/A test first** to validate setup
  - Do an **SRM check** to confirm assignment balance
  - Monitor **guardrail metrics** (e.g., bounce rate, latency)

- ⏳ Add **1–2 buffer days** for these. It’s not just about stats — it’s about reliability and business trust.

---

##### 🧠 Tip
> “We typically recommend calculating sample size from power analysis, then dividing by daily traffic per variant. But we also factor in buffer days for ramp-up, trust checks, and traffic noise — just to make sure we don’t rush analysis before data is stable.”

> “We use power analysis to plan — it gives stakeholders a timeline and sets expectations.
But we don’t blindly stop based on sample size. I monitor SRM, metric stability, cohort coverage, and confidence intervals before making the call. We want decisions that are trustworthy, not just statistically complete.”

In [ ]:
def estimate_total_test_duration(required_sample_size_per_group, daily_eligible_users, allocation_ratio=0.5):
    """
    Estimate total days needed to complete an A/B test for both groups.

    Parameters:
    - required_sample_size_per_group: int, sample size needed per group (from power analysis)
    - daily_eligible_users: int, total eligible users arriving each day
    - allocation_ratio: float, proportion of traffic sent to each group (e.g., 0.5 for 50/50)

    Returns:
    - Estimated total number of days to complete both groups
    """
    # Max group load per day (assuming symmetric or asymmetric allocation)
    daily_users_per_group = daily_eligible_users * allocation_ratio
    days_needed = required_sample_size_per_group / daily_users_per_group

    return int(np.ceil(days_needed))

# Example Usage
required_sample = 3532
daily_users = 10000
allocation = 0.5  # 50% of users per group

total_days = estimate_total_test_duration(required_sample, daily_users, allocation)
buffer_days = 2  # For ramp-up, cool-down, or traffic anomalies

print(f"📅 Estimated minimum duration per group : {total_days} days")
print(f"📦 Add buffer (ramp-up, weekends, trust checks): {buffer_days} days")
print(f"🧮 Total recommended runtime             : {total_days + buffer_days} days\n")


#### Monitoring Dashboard
- Guardrails
-

[Back to the top](#Contents)
___


# Post Hoc Analysis

> 🔍 After statistical significance, we do a post-hoc dive. We check for lift by user segment, inspect guardrail metrics, and simulate rollout impact. This turns A/B tests into real product decisions.

Post-hoc analysis helps interpret test results **beyond the primary metric**, and answer key follow-up questions like:
- Did the effect differ by segment (e.g. platform, user type)?
- Were any guardrail metrics negatively impacted?
- What does the lift mean in terms of actual revenue or conversions?
- What happens if we roll this out 100%?

This step helps turn **statistical significance into business confidence.**

#### Segmented Lift

In [ ]:
# Let's add a few realistic segmentation features to the users DataFrame
# (e.g., platform, device_type, user_tier, region)

np.random.seed(my_seed)  # for reproducibility

users['platform'] = np.random.choice(['iOS', 'Android'], size=len(users))
users['device_type'] = np.random.choice(['mobile', 'desktop'], size=len(users), p=[0.7, 0.3])
users['user_tier'] = np.random.choice(['new', 'returning'], size=len(users), p=[0.4, 0.6])
users['region'] = np.random.choice(['North', 'South', 'East', 'West'], size=len(users))

# Return a preview of the updated DataFrame
users.head()

In [ ]:
def segmented_lift_analysis(df, segment_cols, outcome_col='converted'):
    for segment in segment_cols:
        print(f"\n📊 Segment: {segment}")
        print("-" * 40)

        # 1. Aggregate conversion metrics
        grouped = df.groupby([segment, 'group'])[outcome_col].agg(['count', 'sum', 'mean']).reset_index()
        grouped.columns = [segment, 'group', 'n_users', 'n_converted', 'conversion_rate']

        # 2. Pivot to calculate absolute and percentage lift
        pivoted = grouped.pivot(index=segment, columns='group', values='conversion_rate').reset_index()
        pivoted['absolute_lift'] = pivoted['treatment'] - pivoted['control']
        pivoted['percentage_lift'] = pivoted['absolute_lift'] / pivoted['control']
        print(pivoted[[segment, 'control', 'treatment', 'absolute_lift', 'percentage_lift']].to_string(index=False))

        # 3. Plot
        plt.figure(figsize=(10, 4))
        ax = sns.barplot(data=grouped, x=segment, y='conversion_rate', hue='group')

        # 4. Add value labels
        for bar in ax.patches:
            height = bar.get_height()
            ax.annotate(f"{height:.3f}", 
                        (bar.get_x() + bar.get_width() / 2, height + 0.005),
                        ha='center', va='bottom', fontsize=9)

        # 5. Style tweaks — fixed layout
        ax.set_title(f'Conversion Rate by {segment} and Group', pad=15)
        ax.set_ylabel('Conversion Rate')
        ax.set_xlabel(segment)
        ax.grid(axis='y', linestyle='--', alpha=0.6)
        ax.set_ylim(0, grouped['conversion_rate'].max() + 0.05)

        # 👇 FIXED LEGEND POSITION
        ax.legend(
            # title='Group', 
            loc='upper right', 
            fontsize='small', 
            title_fontsize='small',
            frameon=True,
            borderpad=0.5)

        plt.tight_layout()
        plt.show()

In [ ]:
segmented_columns = ['platform', 'device_type', 'user_tier', 'region']
segmented_lift_analysis(users, segmented_columns)

#### Guardrail Metric Checks

Guardrail metrics are **non-primary metrics** that help ensure an experiment isn't causing unintended harm.

We monitor them alongside the main success metric to:
- Detect regressions in user experience or system performance
- Catch trade-offs early (e.g., improved conversions but worse bounce rate)

---

📊 Common Guardrail Metrics
- Bounce Rate — did the experience frustrate users?
- Page Load Time / Latency — did the feature slow down the UI?
- Session Length / Engagement — did users stick around?
- Error Rate — did the experiment introduce bugs?

---

✅ When to Act:
- If the **treatment worsens a guardrail metric**, even if the primary metric improved → we pause, investigate, or reject the rollout.
Guardrails aren’t always deal-breakers, but they’re **trust checks** that make A/B test results *safe* to act on.

In [ ]:
# Add a simulated guardrail metric: bounce_rate (continuous between 0.1 and 0.9)
# Bounce rate is typically higher when users disengage quickly

np.random.seed(my_seed)
users['bounce_rate'] = np.where(
    users['converted'] == 1,
    np.random.normal(loc=0.2, scale=0.05, size=len(users)),  # lower bounce if converted
    np.random.normal(loc=0.6, scale=0.1, size=len(users))   # higher bounce if not
)

# Clip to stay between 0 and 1
users['bounce_rate'] = users['bounce_rate'].clip(0, 1)

# Return sample for inspection
users.head()

In [ ]:
# Example if you had bounce_rate as a column:
guardrail = users.groupby('group')['bounce_rate'].mean()
print("🚦 Average Bounce Rate by Group:")
print(guardrail)

In [ ]:
# 1. Group means
bounce_by_group = users.groupby('group')['bounce_rate'].mean()
bounce_diff = bounce_by_group['treatment'] - bounce_by_group['control']

# 2. Optional statistical test (t-test)
control_bounce = users[users['group'] == 'control']['bounce_rate']
treatment_bounce = users[users['group'] == 'treatment']['bounce_rate']
t_stat, p_val = ttest_ind(treatment_bounce, control_bounce)

# 3. Print summary and conclusion
print("🚦 Guardrail Check")
print(f"- Control  : {bounce_by_group['control']:.4f}")
print(f"- Treatment: {bounce_by_group['treatment']:.4f}")
print(f"- Difference: {bounce_diff:+.4f}")
print(f"- P-value (t-test): {p_val:.4f}")

# 4. Conclusion
if p_val < 0.05:
    if bounce_diff > 0:
        print("❌ Statistically significant *increase* in bounce rate — potential UX regression.")
    else:
        print("✅ Statistically significant *decrease* in bounce rate — UX may have improved.")
else:
    print("🟡 No significant change in bounce rate — guardrail looks stable.")

#### Rollout Simulation

In [ ]:
# Assume full exposure to daily_eligible_users
daily_impact = (treatment_conv - control_conv) * daily_eligible_users
monthly_impact = daily_impact * 30

print(f"📈 Estimated additional conversions per day: {daily_impact:.0f}")
print(f"📈 Estimated additional conversions per month: {monthly_impact:.0f}")

[Back to the top](#Contents)
___


# Scratch Notes


In [ ]:
# Simulate data
n_control = 5000  # Sample size for control group
n_treatment = 5000  # Sample size for treatment group

# Assume conversion rate is 10% for control and 12% for treatment
control_conversions = np.random.binomial(1, 0.10, n_control)
treatment_conversions = np.random.binomial(1, 0.12, n_treatment)

# Create DataFrame
df = pd.DataFrame({
    'group': ['control'] * n_control + ['treatment'] * n_treatment,
    'conversion': np.concatenate([control_conversions, treatment_conversions])
})
df.head()
df.shape

# Display summary
df.groupby('group')['conversion'].agg(['mean', 'count', 'sum'])


#### AB Testing

1. Chi-Square Test (Categorical Data - Conversion Rates)     
Used when testing differences in proportions (e.g., conversion rate increase).

In [ ]:
# Create contingency table
conversion_table = pd.crosstab(df['group'], df['conversion'])

# Chi-Square Test
chi2_stat, p_value, dof, expected = chi2_contingency(conversion_table)

# Print results
print(f"Chi-Square Statistic: {chi2_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

# Check significance
alpha = 0.05  # 95% confidence level
if p_value < alpha:
    print("Reject the null hypothesis: Significant difference detected.")
else:
    print("Fail to reject the null hypothesis: No significant difference detected.")


2. Independent T-Test (Continuous Data - Avg. Time Spent, Revenue)    
Used when comparing means of continuous data between two independent groups.

In [ ]:
# Perform independent t-test
t_stat, p_val = ttest_ind(control_conversions, treatment_conversions)

print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_val:.4f}")

if p_val < alpha:
    print("Reject the null hypothesis: The new feature has a significant effect.")
else:
    print("Fail to reject the null hypothesis: No significant difference detected.")


3. Paired T-Test (Before/After Tests - Same Users)    
Used when measuring differences within the same group (e.g., before vs. after).

In [ ]:
# Example: User engagement before & after a UI change
before = np.random.normal(200, 25, 100)  # Mean 200 sec, std dev 25
after = np.random.normal(210, 25, 100)  # Mean 210 sec, std dev 25

t_stat, p_val = ttest_rel(before, after)

print(f"Paired T-Test Statistic: {t_stat:.4f}")
print(f"P-Value: {p_val:.4f}")

if p_val < 0.05:
    print("Significant difference detected.")
else:
    print("No significant difference detected.")

4. Mann-Whitney U Test (Non-Normal Continuous Data)
A non-parametric test used when data isn’t normally distributed (e.g., skewed revenue).

In [ ]:
# Example: Revenue data (skewed distribution)
control_revenue = np.random.exponential(50, 5000)  # Skewed
treatment_revenue = np.random.exponential(55, 5000)  # Skewed

u_stat, p_val = mannwhitneyu(control_revenue, treatment_revenue)

print(f"Mann-Whitney U Statistic: {u_stat:.4f}")
print(f"P-Value: {p_val:.4f}")

if p_val < 0.05:
    print("Significant difference detected.")
else:
    print("No significant difference detected.")


 5. Bayesian A/B Testing (Alternative Approach)    
Instead of p-values, Bayesian A/B testing provides posterior probability distributions.

In [ ]:
# import pymc3 as pm

# # Simulated conversion data
# control_conversions = np.sum(np.random.binomial(1, 0.10, 5000))
# treatment_conversions = np.sum(np.random.binomial(1, 0.12, 5000))

# with pm.Model():
#     control_rate = pm.Beta("control_rate", alpha=1, beta=1)
#     treatment_rate = pm.Beta("treatment_rate", alpha=1, beta=1)

#     # Priors
#     control = pm.Binomial("control", n=5000, p=control_rate, observed=control_conversions)
#     treatment = pm.Binomial("treatment", n=5000, p=treatment_rate, observed=treatment_conversions)

#     trace = pm.sample(2000, return_inferencedata=True)

# # Compute probability that the treatment is better
# prob_treatment_better = (trace.posterior["treatment_rate"] > trace.posterior["control_rate"]).mean().item()

# print(f"Probability that the treatment is better: {prob_treatment_better:.4f}")


6. Power Analysis (How Much Data Do You Need?)
Used to determine sample size before running an experiment.

In [ ]:
# Define parameters
effect_size = 0.02  # Expected lift in conversion rate
alpha = 0.05  # Significance level
power = 0.8  # Desired statistical power

# Compute sample size per group
analysis = TTestIndPower()
sample_size = analysis.solve_power(effect_size, power=power, alpha=alpha, ratio=1)

print(f"Required Sample Size per Group: {int(sample_size)}")

# Other Notes

### Experimentation Infrastructure

[Back to the top](#Contents)
___